In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
import numpy as np
import cv2 # extract frames from the videos
from PIL import Image  # to manipulate images
import os

from tensorflow.keras.applications import vgg16,vgg19,DenseNet201
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.applications.imagenet_utils import preprocess_input

from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from google.colab.patches import cv2_imshow
import pickle
from tqdm import tqdm

In [49]:
# load the model
dense_model = DenseNet201(weights='imagenet')

# remove the last layers in order to get features instead of predictions
feat_extractor = Model(inputs=dense_model.input, outputs=dense_model.get_layer("avg_pool").output)

# print the layers of the CNN
feat_extractor.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [50]:
# parameters setup

video_path = "/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/"
target_path = "/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/test_video/"
video_model_width, video_model_height = 224, 224
No_of_frames = 10

nb_closest_videos = 5 # number of most similar images to retrieve

In [51]:
files_train = [video_path + x for x in os.listdir(video_path) if ".mp4" in x]
#files = np.random.choice(files, 5)
files = list(set(files))
print("number of videos:",len(files))

number of videos: 14


In [52]:
files_test = [target_path + x for x in os.listdir(target_path) if ".mp4" in x]
#files1 = np.random.choice(files1, 3)
print("number of images:",len(files1))
files1 = list(files1)
files1

number of images: 1


['/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/test_video/HoV_CreativeWorkshopConcept1Liveaction_V33386_7_English_Portrait&IMG=PR.mp4']

In [53]:
with open("/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/training_data/files.txt","wb") as fp:
  pickle.dump(files, fp)

In [54]:
with open("/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/training_data/files.txt","rb") as fp:
  files = pickle.load(fp)

In [55]:
def video_feature_extract(files):    
    importedvideos = []
    # load all the images and prepare them for feeding into the CNN
    counter = 0

    for video in tqdm(files,desc="Video to array to features"):
        
          frames = []
          count = 1

          path = video
          cap = cv2.VideoCapture(path)

          length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
          print("The total frames in the video is :",length)
          temp = length / No_of_frames

          ret = True
          while ret:
            ret, img = cap.read() # read one frame from the 'capture' object; img is (H, W, C)
            if ret:
                img = cv2.resize(img, (224, 224))
                frames.append(img)
        #       cv2_imshow(img)
        #      print("****************************")
        #      print("****************************")
                count += temp
                cap.set(1,count)
            else:
              cap.release()
              break
          video_frames = np.stack(frames, axis=0)  
          # extract the images features

          imgs_features = feat_extractor.predict(video_frames)

          print("features successfully extracted!")
          print(imgs_features.shape)

          flatten_img_features = imgs_features.flatten()
          flatten_img_features = np.expand_dims(flatten_img_features,axis = 0)
          print(flatten_img_features.shape)
          importedvideos.append(flatten_img_features)
    videos_array = np.vstack(importedvideos)
    return videos_array

In [56]:
videos_train = video_feature_extract(files)
videos_test = video_feature_extract(files1)

Video to array to features:   0%|          | 0/14 [00:00<?, ?it/s]

The total frames in the video is : 300


Video to array to features:   7%|▋         | 1/14 [00:05<01:10,  5.41s/it]

features successfully extracted!
(10, 1920)
(1, 19200)
The total frames in the video is : 900


Video to array to features:  14%|█▍        | 2/14 [00:07<00:52,  4.34s/it]

...

features successfully extracted!
(10, 1920)
(1, 19200)
The total frames in the video is : 300


Video to array to features:   0%|          | 0/1 [00:00<?, ?it/s]

features successfully extracted!
(10, 1920)
(1, 19200)
The total frames in the video is : 210


Video to array to features: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]

features successfully extracted!
(10, 1920)
(1, 19200)


In [57]:
print(videos_train.shape)
print(videos_test.shape)

(14, 19200)
(1, 19200)


In [58]:
np.save("/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/training_data/train.npy", videos_train)

In [59]:
videos_train = np.load("/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/training_data/train.npy")
videos_train.shape

(14, 19200)

In [60]:
arr = np.concatenate((videos_train, videos_test), axis=0)
arr.shape

(15, 19200)

In [61]:
files_total = files_train + files_test
files_total

['/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33400_30_English_Landscape&IMG=.mp4',
 '/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33399_30_English_Portrait&IMG=P.mp4',
 '/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33398_30_English_Portrait&IMG=P.mp4',
 '/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33397_30_English_Square&IMG=PRP.mp4',
 '/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33396_15_English_Portrait&IMG=P.mp4',
 '/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33395_15_English_Landscape&IMG=.mp4',
 '/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33394_15_English_Portrait&IMG=P.mp4',
 '/content/gdrive/My

In [62]:
# compute cosine similarities between videos

cosSimilarities = cosine_similarity(arr)

# store the results into a pandas dataframe

cos_similarities_df = pd.DataFrame(cosSimilarities, columns=files_total, index=files_total)
cos_similarities_df.head()

,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33400_30_English_Landscape&IMG=.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33399_30_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33398_30_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33397_30_English_Square&IMG=PRP.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33396_15_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33395_15_English_Landscape&IMG=.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33394_15_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33393_15_English_Square&IMG=PRL.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33392_10_English_Landscape&IMG=.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33391_10_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33390_10_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33389_10_English_Square&IMG=PRH.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33388_7_English_Landscape&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33387_7_English_Portrait&IMG=PR.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/test_video/HoV_CreativeWorkshopConcept1Liveaction_V33386_7_English_Portrait&IMG=PR.mp4
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33400_30_English_Landscape&IMG=.mp4,1.000001,0.757006,0.852962,0.874217,0.769312,0.784205,0.861444,0.823943,0.915985,0.848973,0.976984,0.920634,0.887727,0.943205,0.895052
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33399_30_English_Portrait&IMG=P.mp4,0.757006,1.000001,0.879819,0.775328,0.817932,0.769025,0.866167,0.934390,0.774702,0.769581,0.780618,0.760962,0.813900,0.772049,0.834089
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33398_30_English_Portrait&IMG=P.mp4,0.852962,0.879819,1.000001,0.823172,0.752519,0.739663,0.976706,0.959958,0.857402,0.755745,0.877846,0.838937,0.851323,0.843401,0.881693
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33397_30_English_Square&IMG=PRP.mp4,0.874217,0.775328,0.823172,1.000000,0.758645,0.899328,0.826786,0.827671,0.961370,0.850560,0.878868,0.950682,0.793780,0.924039,0.812241
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33396_15_English_Portrait&IMG=P.mp4,0.769312,0.817932,0.752519,0.758645,1.000002,0.785334,0.760309,0.778287,0.767691,0.850960,0.779967,0.770760,0.873816,0.786769,0.886437


In [63]:
cos_similarities_df.tail()

,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33400_30_English_Landscape&IMG=.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33399_30_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33398_30_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33397_30_English_Square&IMG=PRP.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33396_15_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33395_15_English_Landscape&IMG=.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33394_15_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33393_15_English_Square&IMG=PRL.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33392_10_English_Landscape&IMG=.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33391_10_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33390_10_English_Portrait&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33389_10_English_Square&IMG=PRH.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33388_7_English_Landscape&IMG=P.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33387_7_English_Portrait&IMG=PR.mp4,/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/test_video/HoV_CreativeWorkshopConcept1Liveaction_V33386_7_English_Portrait&IMG=PR.mp4
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33390_10_English_Portrait&IMG=P.mp4,0.976984,0.780618,0.877846,0.878868,0.779967,0.798679,0.873737,0.850454,0.921945,0.870298,1.000001,0.910348,0.889104,0.954742,0.915247
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33389_10_English_Square&IMG=PRH.mp4,0.920634,0.760962,0.838937,0.950682,0.770760,0.856566,0.850785,0.814168,0.980475,0.838986,0.910348,1.000001,0.837438,0.913305,0.848025
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33388_7_English_Landscape&IMG=P.mp4,0.887727,0.813900,0.851323,0.793780,0.873816,0.751327,0.864358,0.842257,0.832082,0.828374,0.889104,0.837438,1.000002,0.863927,0.977471
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33387_7_English_Portrait&IMG=PR.mp4,0.943205,0.772049,0.843401,0.924039,0.786769,0.836791,0.845813,0.835900,0.923072,0.904404,0.954742,0.913305,0.863927,1.000001,0.882740
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/test_video/HoV_CreativeWorkshopConcept1Liveaction_V33386_7_English_Portrait&IMG=PR.mp4,0.895052,0.834089,0.881693,0.812241,0.886437,0.763063,0.885067,0.869051,0.851786,0.838123,0.915247,0.848025,0.977471,0.882740,1.000002


In [64]:
# function to retrieve the most similar video to the given

def retrieve_most_similar_videos(given_video):

    print("*********************************")
    print("Original Add Video:")
    print("*********************************")
    print(given_video)
    print("-----------------------------------------------------------------------")
    print("Most Similar Video Recommended:")

    closest_videos = cos_similarities_df[given_video].sort_values(ascending=False)[1:nb_closest_videos+1].index
    closest_videos_scores = cos_similarities_df[given_video].sort_values(ascending=False)[1:nb_closest_videos+1]

    for i in range(0,len(closest_videos)):
        print(closest_videos[i])
        print("Cosine Similarity Score: ",closest_videos_scores[i])
        print('---------------------------------------------------------------------')

In [65]:
retrieve_most_similar_videos(files_total[len(files_total)-1])

*********************************
Original Add Video:
*********************************
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/test_video/HoV_CreativeWorkshopConcept1Liveaction_V33386_7_English_Portrait&IMG=PR.mp4
-----------------------------------------------------------------------
Most Similar Video Recommended:
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33388_7_English_Landscape&IMG=P.mp4
Cosine Similarity Score:  0.9774711
---------------------------------------------------------------------
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33390_10_English_Portrait&IMG=P.mp4
Cosine Similarity Score:  0.9152467
---------------------------------------------------------------------
/content/gdrive/MyDrive/BLKBOX_VIDEO_SIMILARITY_MODEL/video/HoV_CreativeWorkshopConcept1Liveaction_V33400_30_English_Landscape&IMG=.mp4
Cosine Similarity Score:  0.89505243
----------